# Here Comes the Sun

Code for showing were batteries are in CAISO, and how they grew over time

In [75]:
#import modules 
import pandas as pd
import datetime as dt 
import matplotlib.pyplot as plt
import holoviews as hv
import geoviews as gv
import hvplot
import hvplot.pandas
import pathlib
import os
from glob import glob
import geopandas as gpd
import requests
import io 
import zipfile
import panel as pn 
import calendar
import warnings




In [9]:
#set up data directory 

map_dir = os.path.join(
    pathlib.Path.home(),
    'code-projects',
    'generation-analysis',
    'map-data'
    )
os.makedirs(map_dir, exist_ok=True)

map_data = os.path.join (map_dir, 'CA-batteries-data.geojson')

In [10]:
# retrieve power plant data 

# Set up URL 
pp_url = (
    "https://services7.arcgis.com/"
    "FGr1D95XCGALKXqM/arcgis/rest/services"
    "/Power_Plants_Testing/FeatureServer"
    "/0/query?where=State%20%3D%20'CALIFORNIA'"
    "%20AND%20PrimSource%20%3D%20'BATTERIES'&outFields=*&outSR=4326&f=json"
)

#download data
if not os.path.exists(map_data):
   #Get File with requests
   gdf=gpd.read_file(pp_url)
   gdf.to_file(map_data, driver="GeoJSON")

pp_gdf = gpd.read_file(map_data)



#caiso_gdf['StartDate'] = pd.to_datetime(caiso_gdf['StartDate'], unit= 'ms')

pp_gdf


,OBJECTID,Plant_Code,Plant_Name,Utility_ID,Utility_Name,sector_name,Street_Address,City,County,State,...,Nuclear_MW,Crude_MW,Solar_MW,Wind_MW,Other_MW,Source,Period,Longitude,Latitude,geometry
0,38652,50300,VESI Pomona,54736,"VESI Pomona Energy Storage, Inc.",IPP Non-CHP,1507 Mount Vernon Ave.,Pomona,Los Angeles,California,...,None,None,NaN,None,None,"EIA-860, EIA-860M and EIA-923",202406,-117.774167,34.059444,POINT (-117.77417 34.05944)
1,43318,60565,Borrego Springs Energy Storage,60336,SDGE Batteries,Electric Utility,2100 Borrego Valley Road,Borrego Springs,San Diego,California,...,None,None,NaN,None,None,"EIA-860, EIA-860M and EIA-923",202406,-116.349600,33.270110,POINT (-116.34960 33.27011)
2,43319,60566,Pala Energy Storage Yard,60336,SDGE Batteries,Electric Utility,HWY 76 & Pala Del Norte,Pala,San Diego,California,...,None,None,NaN,None,None,"EIA-860, EIA-860M and EIA-923",202406,-117.113599,33.356510,POINT (-117.11360 33.35651)
3,43320,60567,Ortega Highway Energy Storage,60336,SDGE Batteries,Electric Utility,33608 Ortega Highway,San Juan Capistrano,Orange,California,...,None,None,NaN,None,None,"EIA-860, EIA-860M and EIA-923",202406,-117.543777,33.562061,POINT (-117.54378 33.56206)
4,43321,60568,Canyon Crest Academy,60336,SDGE Batteries,Electric Utility,5951 Village Center Loop Road,San Diego,San Diego,California,...,None,None,NaN,None,None,"EIA-860, EIA-860M and EIA-923",202406,-117.191622,32.959070,POINT (-117.19162 32.95907)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,48075,66500,Valley Center 85,65552,Terra-Gen Operating Co-BESS 2,IPP Non-CHP,29523 Valley Center Road,Valley Center,San Diego,California,...,None,None,NaN,None,None,"EIA-860, EIA-860M and EIA-923",202406,-117.017744,33.227236,POINT (-117.01774 33.22724)
79,48077,66502,Valley Center 54,65552,Terra-Gen Operating Co-BESS 2,IPP Non-CHP,29523 Valley Center Road,Valley Center,San Diego,California,...,None,None,NaN,None,None,"EIA-860, EIA-860M and EIA-923",202406,-117.017744,33.227236,POINT (-117.01774 33.22724)
80,48116,66575,Sunlight Storage II,65625,Sunlight Storage II,IPP Non-CHP,44810 Kaiser Rd,Desert Center,Riverside,California,...,None,None,NaN,None,None,"EIA-860, EIA-860M and EIA-923",202406,-115.793889,33.809536,POINT (-115.79389 33.80954)
81,48237,66807,SEPV Cuyama Hybrid,65779,"SEPV Cuyama, LLC",IPP Non-CHP,1696 Foothill Rd.,New Cuyama,Santa Barbara,California,...,None,None,1.6,None,None,"EIA-860, EIA-860M and EIA-923",202406,-119.584995,34.904711,POINT (-119.58499 34.90471)


In [29]:
#download EIA Data

# set path
eai_path = os.path.join(map_dir,'eia_files')

#data for EIA Form 860 for 202023
eia_url = (
    "https://www.eia.gov/electricity"
    "/data/eia860/xls/eia8602023.zip"
)

#download and extract data
if not os.path.exists(eai_path):
    response = requests.get(eia_url)
    if response.status_code == 200:
        os.makedirs(eai_path, exist_ok=True)
        with zipfile.ZipFile(io.BytesIO(response.content)) as z:
            z.extractall(eai_path)
        print("Download and extraction complete!")
    else:
        print("Failed to download the file.")
#Grab Energy Storage Spreadsheet
storage_path = os.path.join(eai_path, '3_4_Energy_Storage_Y2023.xlsx')
storage_df = pd.read_excel(storage_path, sheet_name='Operable', header=1)

#Filter so only project from CA are shown 
storage_df = storage_df[storage_df['State'] == 'CA']

#drop duplicates 
storage_df = storage_df.drop_duplicates(subset=["Plant Code"], keep='first')

fil_storage_df = storage_df[['Plant Code', 'Plant Name', 'Operating Month', 'Operating Year', 'Nameplate Capacity (MW)', 'Nameplate Energy Capacity (MWh)', 'Storage Technology 1']]
fil_storage_df = fil_storage_df.rename(columns={'Plant Code':'Plant_Code'})

fil_storage_df

,Plant_Code,Plant Name,Operating Month,Operating Year,Nameplate Capacity (MW),Nameplate Energy Capacity (MWh),Storage Technology 1
1,260,Dynegy Moss Landing Power Plant Hybrid,5,2021,300.0,1200,LIB
4,389,El Centro Hybrid,10,2016,30.0,20,LIB
18,6704,Pebbly Beach Generating Station Hybrid,8,2012,1.0,6.8,NAB
21,10156,Fresno Cogen Partners,3,2023,16.0,16,LIB
23,50064,University of California Santa Cruz,12,2022,1.0,1,LIB
...,...,...,...,...,...,...,...
563,66772,Estrella Solar & Storage,12,2023,28.0,112,LIB
564,66799,Bishop Ranch - BR 1-Y,3,2022,0.7,1.3,LIB
565,66807,SEPV Cuyama Hybrid,12,2023,3.0,12,LIB
579,67093,Mojave 16/17/18 BESS,8,2020,77.0,77,LIB


In [66]:
#join data frame with geo data frame 

gdf_batteries_raw = pp_gdf.merge(fil_storage_df, on="Plant_Code", how= "inner")

#filter out so only 
gdf_batteries = gdf_batteries_raw[['Plant_Name', 'Utility_Name', 'Operating Year','Nameplate Capacity (MW)', 'Nameplate Energy Capacity (MWh)', 'Storage Technology 1', 'geometry']]




In [91]:
# Set up Map 

# Get map bounds
xmin, ymin, xmax, ymax = gdf_batteries.total_bounds

# Create an IntSlider for year selection
year_widget = pn.widgets.IntSlider(
    name="Year",
    start=gdf_batteries["Operating Year"].min(),
    end=gdf_batteries["Operating Year"].max(),
    value=gdf_batteries["Operating Year"].min()
)

# Function to filter and update the plot
def update_plot(selected_year):
    # Select all rows where 'Operating Year' is <= selected year (cumulative)
    filtered_gdf = gdf_batteries[gdf_batteries["Operating Year"] <= selected_year]

    # Generate hvPlot map
    return filtered_gdf.hvplot(
        geo=True,
        tiles="EsriWorldTopo",
        title=f"CA Battery Additions Up to {selected_year}",
        xlim=(xmin, xmax), ylim=(ymin, ymax),
        frame_height=1000,
        frame_width=800
    )

# Bind the function to the slider
battery_plot = pn.bind(update_plot, year_widget)

# Display the slider and interactive plot in a Panel layout
pn.Column(year_widget, battery_plot).servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'0fd4deee-9f8e-478d-8604-7fe1456f36aa': {'version…

UnknownReferenceError: can't resolve reference 'p12912'

UnknownReferenceError: can't resolve reference 'p12912'

UnknownReferenceError: can't resolve reference 'p12912'

UnknownReferenceError: can't resolve reference 'p12912'

UnknownReferenceError: can't resolve reference 'p13126'

UnknownReferenceError: can't resolve reference 'p13126'

UnknownReferenceError: can't resolve reference 'p13126'

UnknownReferenceError: can't resolve reference 'p13126'

In [89]:
pn.Column(year_widget, battery_plot).save("battery_additions.html", embed=True)